In [14]:
import pandas as pd
from getbibleverse import verse_lookup

In [15]:
def store_verses(bible_books, bible_chapters, bible_verses):

    # create empty df to store verses 
    verses_df = pd.DataFrame(columns=["bookname", "chapter", "verse", "text"])
    
    # get verses and store in data frame
    if verse_lookup([book for book in bible_books], [chapter for chapter in bible_chapters], [verse for verse in bible_verses]) is None:
        print("That didn't work.")
    else:
        verses = pd.json_normalize(
            verse_lookup([book for book in bible_books], [chapter for chapter in bible_chapters], [verse for verse in bible_verses])
        )
        verses_df = pd.concat([verses_df, verses], ignore_index=True)

    # combine data frame values
    books = "".join(pd.unique(verses_df["bookname"]))
    chapters = "".join(pd.unique(verses_df["chapter"]))
    verses = "1-10"
    text = "".join(pd.unique(verses_df["text"]))

    return print("{} \n- {} {}:{}".format(text, books, chapters, verses))

In [16]:
store_verses(['Genesis','Ephesians'], ['1','1'], ['1-10', '1-10'])

Cannot find that book.
That didn't work.
 
-  :1-10


In [17]:
def store_verses2(bible_books, bible_chapters, bible_verses):
     # create empty df to store verses 
    verses_df = pd.DataFrame(columns=["bookname", "chapter", "verse", "text"])
    #verses = pd.json_normalize(
    #        verse_lookup(bible_books, bible_chapters, bible_verses)
    #    )
    #verses_df = pd.concat([verses_df, verses], ignore_index=True)

    return print([book for book in bible_books], [chapter for chapter in bible_chapters], [verse for verse in bible_verses])


In [18]:
store_verses2(['Genesis','Ephesians'], ['1','1'], ['1-10', '1-10'])

['Genesis', 'Ephesians'] ['1', '1'] ['1-10', '1-10']
